In [167]:
# Importar las bibliotecas necesarias
import pandas as pd # Para trabajar con dataframes
import numpy as np # Para operaciones y funciones con arrays
import gzip # Para cargar archivo gz
import ast  # Para cargar contenido tipo diccionario dentro de los json
from transformers import pipeline # modelo entrenado para NPL

In [168]:
# Como estamos trabajando en colab y tengo los archivos en el drive importamos la libreria y montamos
from google.colab import drive #para utilizar google drive como fuente de archivos
drive.mount('/content/drive') #montamos la carpeta del drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
games = pd.read_json("/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/steam_games.json.gz", lines=True)

In [ ]:
games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [ ]:
games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'specs', 'price', 'early_access', 'id',
       'developer'],
      dtype='object')

In [ ]:
games.drop(columns=['publisher', 'genres', 'app_name', 'reviews_url', 'early_access'], inplace=True)

In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         30085 non-null  object 
 1   url           32135 non-null  object 
 2   release_date  30068 non-null  object 
 3   tags          31972 non-null  object 
 4   specs         31465 non-null  object 
 5   price         30758 non-null  object 
 6   id            32133 non-null  float64
 7   developer     28836 non-null  object 
dtypes: float64(1), object(7)
memory usage: 7.4+ MB


In [ ]:
games.dropna(subset='id', inplace=True)

In [ ]:
games.columns

Index(['title', 'url', 'release_date', 'tags', 'specs', 'price', 'id',
       'developer'],
      dtype='object')

In [ ]:
games.drop(columns=['title', 'url', 'specs', 'price', 'developer'], inplace=True)

In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32133 entries, 88310 to 120444
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   release_date  30067 non-null  object 
 1   tags          31971 non-null  object 
 2   id            32133 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1004.2+ KB


In [ ]:
games[games['release_date'].isna()]

,release_date,tags,id
88314,None,"[Action, Indie, Casual, Sports]",773570.0
88321,None,[Casual],724910.0
88329,None,"[Early Access, Indie, VR]",772590.0
88330,None,"[Early Access, Action, Adventure, Indie, Casual]",640250.0
88332,None,"[Early Access, Strategy, Action, Indie, Casual...",711440.0
...,...,...,...
120387,None,"[Early Access, Casual]",705860.0
120394,None,"[Casual, Indie, Action]",729640.0
120395,None,"[Video Production, Utilities, Web Publishing]",755540.0
120398,None,"[Early Access, Action, Adventure, Indie]",708070.0


In [ ]:
games.isna().sum()/len(games)

release_date    0.064295
tags            0.005042
id              0.000000
dtype: float64

In [ ]:
games.dropna(inplace=True)

In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29906 entries, 88310 to 120443
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   release_date  29906 non-null  object 
 1   tags          29906 non-null  object 
 2   id            29906 non-null  float64
dtypes: float64(1), object(2)
memory usage: 934.6+ KB


In [ ]:
games['release_date']

88310     2018-01-04
88311     2018-01-04
88312     2017-07-24
88313     2017-12-07
88315     2018-01-04
             ...    
120439    2018-01-04
120440    2018-01-04
120441    2018-01-04
120442    2018-01-04
120443    2017-09-02
Name: release_date, Length: 29906, dtype: object

In [ ]:
games['release_date'] = games['release_date'].str.extract(r'(\d{4})')

In [ ]:
games['release_date'].isna().sum()

101

In [ ]:
games.dropna(inplace=True)

In [ ]:
games['release_date'] = games['release_date'].astype(int)

In [ ]:
games['release_date'].min()

1970

In [ ]:
games[games['release_date']>2024]

,release_date,tags,id
101738,5275,"[Early Access, Casual, Indie]",710190.0


In [ ]:
games = games[games['release_date'] <= 2023]

In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29804 entries, 88310 to 120443
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   release_date  29804 non-null  int64  
 1   tags          29804 non-null  object 
 2   id            29804 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 931.4+ KB


In [ ]:
games.drop_duplicates(subset='id', inplace=True)

In [ ]:
games['id'] = games['id'].astype(int)

In [ ]:
games.rename(columns={'tags': 'genres'}, inplace=True)

In [ ]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29803 entries, 88310 to 120443
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   release_date  29803 non-null  int64 
 1   genres        29803 non-null  object
 2   id            29803 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 931.3+ KB


In [ ]:
games.to_csv("/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/gamesv1.csv", index=False)

In [179]:
games = pd.read_csv("/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/gamesv1.csv")

In [180]:
games.head()

,release_date,genres,id
0,2018,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",761140
1,2018,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...",643980
2,2017,"['Free to Play', 'Simulation', 'Sports', 'Casu...",670290
3,2017,"['Action', 'Adventure', 'Casual']",767400
4,2018,"['Action', 'Adventure', 'Simulation', 'FPS', '...",772540


In [181]:
games['genres'] = games['genres'].apply(eval)
games=games.explode('genres')

In [182]:
games.head()

,release_date,genres,id
0,2018,Strategy,761140
0,2018,Action,761140
0,2018,Indie,761140
0,2018,Casual,761140
0,2018,Simulation,761140


In [187]:
dummies = pd.get_dummies(games['genres'])
dummies

,1980s,1990's,2.5D,2D,2D Fighter,3D Platformer,3D Vision,4 Player Local,4X,6DOF,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29802,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [188]:
games = pd.concat([games, dummies], axis=1)
games.drop('genres', axis=1, inplace=True)

In [189]:
games.head()

,release_date,id,1980s,1990's,2.5D,2D,2D Fighter,3D Platformer,3D Vision,4 Player Local,...,Warhammer 40K,Web Publishing,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,2018,761140,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,2018,761140,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,2018,761140,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,2018,761140,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,2018,761140,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [190]:
df = games.drop(columns=['id','release_date'])

In [191]:
df.sum().sort_values(ascending=False)

Indie            16294
Action           11980
Adventure         9194
Casual            8842
Strategy          7333
                 ...  
Transhumanism        6
Bowling              5
Cycling              5
Voice Control        4
Faith                4
Length: 337, dtype: int64

In [196]:
df.sum().mean()

468.3234421364985

In [197]:
lcol=df.sum()[df.sum()>df.sum().mean()]

In [198]:
lcol = lcol.index

In [199]:
lcol

Index(['2D', 'Action', 'Adventure', 'Anime', 'Arcade', 'Atmospheric', 'Casual',
       'Classic', 'Co-op', 'Comedy', 'Cute', 'Design & Illustration',
       'Difficult', 'Early Access', 'Exploration', 'FPS', 'Family Friendly',
       'Fantasy', 'Female Protagonist', 'First-Person', 'Free to Play',
       'Funny', 'Gore', 'Great Soundtrack', 'Hidden Object', 'Horror', 'Indie',
       'Local Co-Op', 'Local Multiplayer', 'Massively Multiplayer', 'Movie',
       'Multiplayer', 'Online Co-Op', 'Open World', 'Pixel Graphics',
       'Platformer', 'Point & Click', 'Puzzle', 'RPG', 'RPGMaker', 'Racing',
       'Retro', 'Rogue-like', 'Sandbox', 'Sci-fi', 'Shoot 'Em Up', 'Shooter',
       'Simulation', 'Singleplayer', 'Space', 'Sports', 'Story Rich',
       'Strategy', 'Survival', 'Tactical', 'Third Person', 'Turn-Based',
       'Utilities', 'Violent', 'Visual Novel', 'Zombies'],
      dtype='object')

In [200]:
l1=['id','release_date']
l2=lcol.to_list()

In [201]:
l1

['id', 'release_date']

In [202]:
l2

['2D',
 'Action',
 'Adventure',
 'Anime',
 'Arcade',
 'Atmospheric',
 'Casual',
 'Classic',
 'Co-op',
 'Comedy',
 'Cute',
 'Design & Illustration',
 'Difficult',
 'Early Access',
 'Exploration',
 'FPS',
 'Family Friendly',
 'Fantasy',
 'Female Protagonist',
 'First-Person',
 'Free to Play',
 'Funny',
 'Gore',
 'Great Soundtrack',
 'Hidden Object',
 'Horror',
 'Indie',
 'Local Co-Op',
 'Local Multiplayer',
 'Massively Multiplayer',
 'Movie',
 'Multiplayer',
 'Online Co-Op',
 'Open World',
 'Pixel Graphics',
 'Platformer',
 'Point & Click',
 'Puzzle',
 'RPG',
 'RPGMaker',
 'Racing',
 'Retro',
 'Rogue-like',
 'Sandbox',
 'Sci-fi',
 "Shoot 'Em Up",
 'Shooter',
 'Simulation',
 'Singleplayer',
 'Space',
 'Sports',
 'Story Rich',
 'Strategy',
 'Survival',
 'Tactical',
 'Third Person',
 'Turn-Based',
 'Utilities',
 'Violent',
 'Visual Novel',
 'Zombies']

In [203]:
cols=l1+l2

In [204]:
cols

['id',
 'release_date',
 '2D',
 'Action',
 'Adventure',
 'Anime',
 'Arcade',
 'Atmospheric',
 'Casual',
 'Classic',
 'Co-op',
 'Comedy',
 'Cute',
 'Design & Illustration',
 'Difficult',
 'Early Access',
 'Exploration',
 'FPS',
 'Family Friendly',
 'Fantasy',
 'Female Protagonist',
 'First-Person',
 'Free to Play',
 'Funny',
 'Gore',
 'Great Soundtrack',
 'Hidden Object',
 'Horror',
 'Indie',
 'Local Co-Op',
 'Local Multiplayer',
 'Massively Multiplayer',
 'Movie',
 'Multiplayer',
 'Online Co-Op',
 'Open World',
 'Pixel Graphics',
 'Platformer',
 'Point & Click',
 'Puzzle',
 'RPG',
 'RPGMaker',
 'Racing',
 'Retro',
 'Rogue-like',
 'Sandbox',
 'Sci-fi',
 "Shoot 'Em Up",
 'Shooter',
 'Simulation',
 'Singleplayer',
 'Space',
 'Sports',
 'Story Rich',
 'Strategy',
 'Survival',
 'Tactical',
 'Third Person',
 'Turn-Based',
 'Utilities',
 'Violent',
 'Visual Novel',
 'Zombies']

In [205]:
games=games[cols]

In [206]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 157825 entries, 0 to 29802
Data columns (total 63 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   id                     157825 non-null  int64
 1   release_date           157825 non-null  int64
 2   2D                     157825 non-null  uint8
 3   Action                 157825 non-null  uint8
 4   Adventure              157825 non-null  uint8
 5   Anime                  157825 non-null  uint8
 6   Arcade                 157825 non-null  uint8
 7   Atmospheric            157825 non-null  uint8
 8   Casual                 157825 non-null  uint8
 9   Classic                157825 non-null  uint8
 10  Co-op                  157825 non-null  uint8
 11  Comedy                 157825 non-null  uint8
 12  Cute                   157825 non-null  uint8
 13  Design & Illustration  157825 non-null  uint8
 14  Difficult              157825 non-null  uint8
 15  Early Access      

In [207]:
games.to_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/gamesv3.parquet', index=False)

In [208]:
games = pd.read_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/gamesv3.parquet')

In [209]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157825 entries, 0 to 157824
Data columns (total 63 columns):
 #   Column                 Non-Null Count   Dtype
---  ------                 --------------   -----
 0   id                     157825 non-null  int64
 1   release_date           157825 non-null  int64
 2   2D                     157825 non-null  uint8
 3   Action                 157825 non-null  uint8
 4   Adventure              157825 non-null  uint8
 5   Anime                  157825 non-null  uint8
 6   Arcade                 157825 non-null  uint8
 7   Atmospheric            157825 non-null  uint8
 8   Casual                 157825 non-null  uint8
 9   Classic                157825 non-null  uint8
 10  Co-op                  157825 non-null  uint8
 11  Comedy                 157825 non-null  uint8
 12  Cute                   157825 non-null  uint8
 13  Design & Illustration  157825 non-null  uint8
 14  Difficult              157825 non-null  uint8
 15  Early Access     

In [210]:
games[['id','release_date','Action']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157825 entries, 0 to 157824
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   id            157825 non-null  int64
 1   release_date  157825 non-null  int64
 2   Action        157825 non-null  uint8
dtypes: int64(2), uint8(1)
memory usage: 2.6 MB


In [211]:
def parseo (ruta):
  # Lista para almacenar los diccionarios JSON
  fjson = []
  # Abre el archivo comprimido en modo lectura binaria ("rb")
  with gzip.open(ruta, "rb") as archivo:
      # Descomprime el contenido del archivo y lo decodifica como utf-8
      contenido = archivo.read().decode("utf-8").splitlines()

      # Recorre cada línea descomprimida
      for fila in contenido:
          # Lee cada línea del archivo como una cadena JSON
          fila_json = ast.literal_eval(fila.strip())

          # Agrega el diccionario JSON a la lista
          fjson.append(fila_json)

  # Crea el DataFrame directamente desde la lista de diccionarios
  df = pd.DataFrame(fjson)
  return df

In [ ]:
reviews = parseo('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/user_reviews.json.gz')

In [ ]:
reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [ ]:
# Utiliza la función apply para aplicar la transformación a cada fila
users_reviews_desanidados = reviews.apply(
    lambda row: [
        {
            'user_id': row['user_id'],
            'funny': r.get('funny', ''),
            'posted': r.get('posted', ''),
            'last_edited': r.get('last_edited', ''),
            'item_id': r.get('item_id', ''),
            'helpful': r.get('helpful', ''),
            'recommend': r.get('recommend', bool),
            'review': r.get('review', '')
        }
        for r in row['reviews']
    ],
    axis=1
)

# Concatena la lista de diccionarios en una lista plana
des_reviews = [review for sublist in users_reviews_desanidados for review in sublist]

# Crea un nuevo DataFrame a partir de la lista des_reviews
users_reviews_optimizado = pd.DataFrame(des_reviews)

# Muestra las primeras filas del DataFrame resultante
users_reviews_optimizado.head(3)


,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [ ]:
users_reviews_optimizado.to_json("/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/reviews.json")

In [ ]:
reviews = pd.read_json("/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/reviews.json")

In [ ]:
reviews.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [ ]:
reviews.columns

Index(['funny', 'posted', 'last_edited', 'item_id', 'helpful', 'recommend',
       'review', 'user_id'],
      dtype='object')

In [ ]:
reviews.drop(columns=['funny', 'last_edited', 'helpful'], inplace=True)

In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59305 entries, 0 to 59304
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   posted     59305 non-null  object
 1   item_id    59305 non-null  int64 
 2   recommend  59305 non-null  bool  
 3   review     59305 non-null  object
 4   user_id    59305 non-null  object
dtypes: bool(1), int64(1), object(3)
memory usage: 2.3+ MB


In [ ]:
df=reviews.head().copy()

In [ ]:
df['posted'] = df['posted'].str.strip('.')

In [ ]:
df['posted'].apply(lambda x: x[-4:])

0    2011
1    2011
2    2011
3    2014
4    2013
Name: posted, dtype: object

In [ ]:
reviews['posted'] = reviews['posted'].str.strip('.')

In [ ]:
reviews['posted'] = reviews['posted'].str.replace(r'[^0-9\'-]', '', regex=True)

In [ ]:
reviews.tail()

,posted,item_id,recommend,review,user_id
59300,10,70,True,a must have classic from steam definitely wort...,76561198312638244
59301,8,362890,True,this game is a perfect remake of the original ...,76561198312638244
59302,3,273110,True,had so much fun plaing this and collecting res...,LydiaMorley
59303,20,730,True,:D,LydiaMorley
59304,2,440,True,so much fun :D,LydiaMorley


In [ ]:
reviews['posted']=reviews['posted'].apply(lambda x: x[-4:])

In [ ]:
reviews['posted'] = reviews['posted'].astype(int)

In [ ]:
reviews[reviews['posted']<2007]

,posted,item_id,recommend,review,user_id
6,3,248820,True,A suitably punishing roguelike platformer. Wi...,evcentric
27,20,730,True,ZIKA DO BAILE,76561198079601835
28,24,730,True,BEST GAME IN THE BLOODY WORLD,MeaTCompany
31,16,252950,True,love it,76561198156664158
32,11,440,True,mt bom,76561198077246154
...,...,...,...,...,...
59300,10,70,True,a must have classic from steam definitely wort...,76561198312638244
59301,8,362890,True,this game is a perfect remake of the original ...,76561198312638244
59302,3,273110,True,had so much fun plaing this and collecting res...,LydiaMorley
59303,20,730,True,:D,LydiaMorley


In [ ]:
dfg = games[['id','release_date']].copy()

In [ ]:
# Función para actualizar los valores de 'posted'
def update_posted(row):
    if row['posted'] < 2007:
        # Buscar el valor correspondiente en dfg basado en 'item_id' e 'id'
        matching_row = dfg[dfg['id'] == row['item_id']]

        # Si hay una coincidencia, devolver el valor de 'release_date'
        if not matching_row.empty:
            return matching_row['release_date'].iloc[0]

    # Si no cumple la condición o no hay coincidencia, mantener el valor original
    return row['posted']

In [ ]:
reviews['posted'] = reviews.apply(update_posted, axis=1)

In [ ]:
reviews['posted'].min()

1

In [ ]:
reviews['posted'].max()

2017

In [ ]:
reviews=reviews[reviews['posted']>=2007]

In [ ]:
reviews.head()

,posted,item_id,recommend,review,user_id
0,2011,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,2011,22200,True,It's unique and worth a playthrough.,76561197970982479
2,2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,2014,251610,True,I know what you think when you see this title ...,js41637
4,2013,227300,True,For a simple (it's actually not all that simpl...,js41637


In [ ]:
# Cargar el modelo de análisis de sentimiento en inglés
sentiment_analyzer = pipeline('sentiment-analysis')

# Función para realizar el análisis de sentimiento
def analyze_sentiment(review):
    try:
        # Utilizar el modelo de análisis de sentimiento
        result = sentiment_analyzer(review)

        # Mapear el resultado a la escala especificada
        sentiment_score = result[0]['label']
        if sentiment_score == 'NEGATIVE':
            return 0
        elif sentiment_score == 'POSITIVE':
            return 2
        else:
            return 1  # Neutral
    except:
        return 1  # En caso de error o reseña ausente, asignar neutral

# Aplicar la función a la columna y crear la nueva columna 'sentiment_analysis'
reviews['sentiment_analysis'] = reviews['review'].apply(analyze_sentiment)

# Eliminar la columna original 'user_reviews.review' si es necesario
reviews.drop('review', axis=1, inplace=True)
reviews.to_parquet("/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/reviews.parquet", index=False)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (588 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
reviews = pd.read_parquet("/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/reviews.parquet")

In [ ]:
reviews.head()

,posted,item_id,recommend,user_id,sentiment_analysis
0,2011,1250,True,76561197970982479,2
1,2011,22200,True,76561197970982479,2
2,2011,43110,True,76561197970982479,2
3,2014,251610,True,js41637,2
4,2013,227300,True,js41637,2


In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57556 entries, 0 to 57555
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   posted              57556 non-null  int64 
 1   item_id             57556 non-null  int64 
 2   recommend           57556 non-null  bool  
 3   user_id             57556 non-null  object
 4   sentiment_analysis  57556 non-null  int64 
dtypes: bool(1), int64(3), object(1)
memory usage: 1.8+ MB


In [ ]:
items = parseo("/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/users_items.json.gz")

In [ ]:
items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [ ]:
items['items'][0]

[{'item_id': '10',
  'item_name': 'Counter-Strike',
  'playtime_forever': 6,
  'playtime_2weeks': 0},
 {'item_id': '20',
  'item_name': 'Team Fortress Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '30',
  'item_name': 'Day of Defeat',
  'playtime_forever': 7,
  'playtime_2weeks': 0},
 {'item_id': '40',
  'item_name': 'Deathmatch Classic',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '50',
  'item_name': 'Half-Life: Opposing Force',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '60',
  'item_name': 'Ricochet',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '70',
  'item_name': 'Half-Life',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '130',
  'item_name': 'Half-Life: Blue Shift',
  'playtime_forever': 0,
  'playtime_2weeks': 0},
 {'item_id': '300',
  'item_name': 'Day of Defeat: Source',
  'playtime_forever': 4733,
  'playtime_2weeks': 0},
 {'item_id': '240',
  'item_name': 'Counter-Strike: S

In [ ]:
items.head(1)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


In [ ]:
pd.json_normalize(items.head(1).to_dict('records'), 'items', ['user_id'])

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6,0,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,7,0,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479
...,...,...,...,...,...
272,289130,Endless Legend,593,0,76561197970982479
273,268500,XCOM 2,1495,0,76561197970982479
274,461640,Sins Of The Demon RPG,0,0,76561197970982479
275,379720,DOOM,793,0,76561197970982479


In [ ]:
items.to_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/items.parquet', index=False)

In [ ]:
items = pd.read_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/items.parquet')

In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [ ]:
items.columns

Index(['user_id', 'items_count', 'steam_id', 'user_url', 'items'], dtype='object')

In [ ]:
items.drop(columns=['items_count', 'steam_id', 'user_url'], inplace=True)

In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  88310 non-null  object
 1   items    88310 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [ ]:
df = items.head(10).copy()

In [ ]:
df

,user_id,items
0,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
5,MinxIsBetterThanPotatoes,"[{'item_id': '50', 'item_name': 'Half-Life: Op..."
6,NitemarePK,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
7,themanwich,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
8,maplemage,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
9,Wackky,[]


In [ ]:
df['items'][0]

array([{'item_id': '10', 'item_name': 'Counter-Strike', 'playtime_2weeks': 0, 'playtime_forever': 6},
       {'item_id': '20', 'item_name': 'Team Fortress Classic', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '30', 'item_name': 'Day of Defeat', 'playtime_2weeks': 0, 'playtime_forever': 7},
       {'item_id': '40', 'item_name': 'Deathmatch Classic', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '50', 'item_name': 'Half-Life: Opposing Force', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '60', 'item_name': 'Ricochet', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '70', 'item_name': 'Half-Life', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '130', 'item_name': 'Half-Life: Blue Shift', 'playtime_2weeks': 0, 'playtime_forever': 0},
       {'item_id': '300', 'item_name': 'Day of Defeat: Source', 'playtime_2weeks': 0, 'playtime_forever': 4733},
       {'item_id': '240', 'item_name': 'Counter-Strik

In [ ]:
df1 = pd.json_normalize(df['items'][0])

In [ ]:
df1

,item_id,item_name,playtime_2weeks,playtime_forever
0,10,Counter-Strike,0,6
1,20,Team Fortress Classic,0,0
2,30,Day of Defeat,0,7
3,40,Deathmatch Classic,0,0
4,50,Half-Life: Opposing Force,0,0
...,...,...,...,...
272,289130,Endless Legend,0,593
273,268500,XCOM 2,0,1495
274,461640,Sins Of The Demon RPG,0,0
275,379720,DOOM,0,793


In [ ]:
df1['user_id']=df['user_id'][0]

In [ ]:
df1

,item_id,item_name,playtime_2weeks,playtime_forever,user_id
0,10,Counter-Strike,0,6,76561197970982479
1,20,Team Fortress Classic,0,0,76561197970982479
2,30,Day of Defeat,0,7,76561197970982479
3,40,Deathmatch Classic,0,0,76561197970982479
4,50,Half-Life: Opposing Force,0,0,76561197970982479
...,...,...,...,...,...
272,289130,Endless Legend,0,593,76561197970982479
273,268500,XCOM 2,0,1495,76561197970982479
274,461640,Sins Of The Demon RPG,0,0,76561197970982479
275,379720,DOOM,0,793,76561197970982479


In [ ]:
df1.drop(df1.index,inplace=True)

In [ ]:
df1

,item_id,item_name,playtime_2weeks,playtime_forever,user_id


In [ ]:
for _, row in df.iterrows():
  df2 = pd.json_normalize(row['items'])
  df2['user_id'] = row['user_id']
  df1 = pd.concat([df1,df2], ignore_index=True)

In [ ]:
df1

,item_id,item_name,playtime_2weeks,playtime_forever,user_id
0,3830,Psychonauts,0.0,333.0,76561197970982479
1,2630,Call of Duty 2,0.0,75.0,76561197970982479
2,3900,Sid Meier's Civilization IV,0.0,338.0,76561197970982479
3,34440,Sid Meier's Civilization IV,0.0,0.0,76561197970982479
4,3920,Sid Meier's Pirates!,0.0,2.0,76561197970982479
...,...,...,...,...,...
3995,207140,SpeedRunners,0.0,0.0,maplemage
3996,369200,Ghost in the Shell Stand Alone Complex First A...,0.0,0.0,maplemage
3997,485890,Summer Sale,0.0,0.0,maplemage
3998,214850,GameMaker: Studio,6353.0,37077.0,maplemage


In [ ]:
for _, row in items.iterrows():
  df2 = pd.json_normalize(row['items'])
  df2['user_id'] = row['user_id']
  df1 = pd.concat([df1,df2], ignore_index=True)
df1.to_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/itemso.parquet', index=False)

In [ ]:
items = pd.read_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/itemso.parquet')

In [ ]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           object 
 1   item_name         object 
 2   playtime_2weeks   float64
 3   playtime_forever  float64
 4   user_id           object 
dtypes: float64(2), object(3)
memory usage: 196.6+ MB


In [ ]:
items.head()

,item_id,item_name,playtime_2weeks,playtime_forever,user_id
0,10,Counter-Strike,0.0,6.0,76561197970982479
1,20,Team Fortress Classic,0.0,0.0,76561197970982479
2,30,Day of Defeat,0.0,7.0,76561197970982479
3,40,Deathmatch Classic,0.0,0.0,76561197970982479
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479


In [ ]:
items.drop(columns=['playtime_2weeks'], inplace=True)

In [ ]:
items['item_id'] = items['item_id'].astype(int)

In [ ]:
items.head(3)

,item_id,item_name,playtime_forever,user_id
0,10,Counter-Strike,6.0,76561197970982479
1,20,Team Fortress Classic,0.0,76561197970982479
2,30,Day of Defeat,7.0,76561197970982479


In [ ]:
items.to_parquet('/content/drive/MyDrive/PI_ML_OPS-PT/Datasets/itemsv2.parquet', index=False)